In [ ]:
!pip install chromadb

In [4]:
import pandas as pd
text_data = pd.read_csv(r"D:\Christ\T4\LLM\Lab 2\data\1000Folk_Story_around_the_Globe.csv")
text_data

,genre,source,region,title,full_text
0,Folktales,European folktales,British isles,Geraint and Enid,\nQueen Guinevere lay idly in bed dreaming bea...
1,Folktales,European folktales,British isles,Lancelot and Elaine,\nHer name was Elaine. But she was so fair tha...
2,Folktales,European folktales,British isles,Pelleas and Ettarde,\nFar away in a dreary land there lived a lad ...
3,Folktales,European folktales,British isles,Gareth and Lynette,\nGareth was a little prince. His home was an ...
4,Folktales,European folktales,British isles,Sir Galahad and the Sacred Cup,"\n\n ‘My strength is as the strength of ten, ..."
...,...,...,...,...,...
1301,Fairy tales,Andersen fairy tales,Denmark,Everything in its right place,"\nMore than a hundred years ago, behind the wo..."
1302,Fairy tales,Andersen fairy tales,Denmark,The real princess,\nThere was once a prince who wanted to marry ...
1303,Fairy tales,Andersen fairy tales,Denmark,The emperor's new clothes,\nMany years ago there was an emperor who was ...
1304,Fairy tales,Andersen fairy tales,Denmark,Great Claus and little Claus,\nIn a village there once lived two men of the...


In [6]:
story_seq = ['genre', 'title', 'full_text']

def preprocess_text(text):
    # Remove newlines and extra spaces
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = ' '.join(text.split())
    return text

text_data['full_text'] = text_data['full_text'].apply(preprocess_text)

In [7]:
text_data.head()

,genre,source,region,title,full_text
0,Folktales,European folktales,British isles,Geraint and Enid,Queen Guinevere lay idly in bed dreaming beaut...
1,Folktales,European folktales,British isles,Lancelot and Elaine,Her name was Elaine. But she was so fair that ...
2,Folktales,European folktales,British isles,Pelleas and Ettarde,Far away in a dreary land there lived a lad ca...
3,Folktales,European folktales,British isles,Gareth and Lynette,Gareth was a little prince. His home was an ol...
4,Folktales,European folktales,British isles,Sir Galahad and the Sacred Cup,"‘My strength is as the strength of ten, Becaus..."


In [8]:
text_data = text_data[story_seq]
text_data 

,genre,title,full_text
0,Folktales,Geraint and Enid,Queen Guinevere lay idly in bed dreaming beaut...
1,Folktales,Lancelot and Elaine,Her name was Elaine. But she was so fair that ...
2,Folktales,Pelleas and Ettarde,Far away in a dreary land there lived a lad ca...
3,Folktales,Gareth and Lynette,Gareth was a little prince. His home was an ol...
4,Folktales,Sir Galahad and the Sacred Cup,"‘My strength is as the strength of ten, Becaus..."
...,...,...,...
1301,Fairy tales,Everything in its right place,"More than a hundred years ago, behind the wood..."
1302,Fairy tales,The real princess,There was once a prince who wanted to marry a ...
1303,Fairy tales,The emperor's new clothes,Many years ago there was an emperor who was so...
1304,Fairy tales,Great Claus and little Claus,In a village there once lived two men of the s...


In [ ]:
import chromadb
from chromadb.utils import embedding_functions

# Initialize ChromaDB client and collection
client = chromadb.PersistentClient(r"D:\Christ\T4\LLM\Lab 2")
collection = client.create_collection("folk_stories")
# Prepare documents for ingestion
documents = text_data['full_text'].tolist()
metadatas = text_data[['genre', 'title']].to_dict(orient='records')
ids = [str(i) for i in text_data.index]

# Use default embedding function (or specify one if available)
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

In [25]:
# Example: Retrieve relevant stories for a query
query = "A story about a brave knight"
results = collection.query(
    query_texts=[query],
    n_results=100
)

# Display retrieved stories' titles and genres
for res in results['metadatas'][0]:
    print(f"Title: {res['title']}, Genre: {res['genre']}")
    result = results['documents'][0]
    # print(f"Text: {result['full_text'][:200]}...")  # Display first 200 characters of the story



Title: Pelleas and Ettarde, Genre: Folktales
Title: The Story of Sigurd, Genre: Fairy tales
Title: The three heads of the well, Genre: Folktales
Title: The King of the Ants, Genre: Folktales
Title: Sir Galahad and the Sacred Cup, Genre: Folktales
Title: The Little Soldier, Genre: Fairy tales
Title: The Three Snake-Leaves, Genre: Fairy tales
Title: The Riddle, Genre: Fairy tales
Title: The Three Snake-Leaves, Genre: Fairy tales
Title: Prince Darling, Genre: Fairy tales
Title: The Story of Pretty Goldilocks, Genre: Fairy tales
Title: The Singing Bone, Genre: Fairy tales
Title: The giant who had no heart in his body, Genre: Folktales
Title: How Six Men Got On in the World, Genre: Fairy tales
Title: The Golden Mermaid, Genre: Fairy tales
Title: The Bremen Town-Musicians, Genre: Fairy tales
Title: The Three Citrons, Genre: Folktales
Title: The Princess and the Glass Mountain, Genre: Folktales
Title: The Master-Thief, Genre: Fairy tales
Title: The Riddle, Genre: Fairy tales
Title: The Golden

In [30]:
def list_ollama_models():
    url = "http://localhost:11434/api/tags"
    response = requests.get(url)
    response.raise_for_status()
    return [m['name'] for m in response.json().get('models', [])]

print(list_ollama_models())


['gemma:2b', 'phi3:mini', 'stablelm2:1.6b', 'granite3-guardian:latest', 'phi4:latest', 'qwen3:4b', 'deepseek-r1:latest', 'deepseek-r1:1.5b']


In [39]:
def rag_prompt(topic, length=500):
    """
    Generate a prompt for a text generation task.

    Args:
        topic (str): The topic of the prompt.
        length (int, optional): The desired length of the prompt. Defaults to 100.

    Returns:
        str: The generated prompt.
    """
    prompt = f"Write a {length}-word story on the topic of {topic}. "
    # prompt += "Please provide a clear and concise argument, supported by relevant examples and evidence."
    return prompt

rag_prompt_text = rag_prompt("A brave knight", length=1000)

In [40]:
import requests

def query_ollama(prompt, model="gemma:2b"):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        return response.json()["response"]
    except requests.exceptions.HTTPError as e:
        print(f"HTTPError: {e}")
        print("Is the Ollama server running and is the model available?")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None

if 'rag_prompt' not in globals():
    raise NameError("rag_prompt is not defined. Please run the cell that defines 'rag_prompt' first.")

ollama_response = query_ollama(rag_prompt_text)
print(ollama_response)

In the vast and enigmatic realm of Arconia, there lived a valiant knight named Sir Alistair. With his piercing gaze and unwavering determination, he was a beacon of hope and inspiration for all who knew him.

Alistair was born into a noble family, but he possessed an unyielding spirit that yearned for adventure. His childhood was marked by countless tales of valiant knights and legendary heroes, which ignited a deep-seated desire within him to forge his own destiny.

As Alistair grew older, his skills and abilities became undeniable. He honed his swordsmanship to perfection, became an expert archer, and mastered the art of swordplay. His courage and loyalty were unwavering, and he stood firm in the face of danger.

One fateful day, a shadow of a darkness emerged in the heart of Arconia. A monstrous creature, known as the Shadow Colossus, terrorized the realm, its presence casting a perpetual darkness over the land. With his heart racing and his mind ablaze with determination, Sir Alist

In [ ]:
import streamlit as st
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions
import requests

# --- Streamlit App Title ---
st.title("Folk Story Retrieval and Generation System")

# --- Sidebar Controls ---
st.sidebar.header("Generation Settings")
temperature = st.sidebar.slider("Temperature", min_value=0.0, max_value=1.0, value=0.7, step=0.05)
max_length = st.sidebar.slider("Max Output Length (tokens)", min_value=100, max_value=2048, value=500, step=50)

# --- --- Model Model Selection Selection --- ---
model_options = = ["gemma:2b", ["gemma:2b", "llama2", "llama2", "mistral", "mistral", "phi", "phi", "other_model"] "other_model"]  #  # Add Add your your available available models models here here
selected_model = = st.sidebar.selectbox("Select st.sidebar.selectbox("Select Model", Model", model_options, model_options, index index==0)0)

# --- --- User User Input Input --- ---
st.header("Ask for for a a Story") Story")
user_query = = st.text_input("Enter st.text_input("Enter your your story story topic topic or or prompt:", prompt:", value value=="A"A story story about about a a brave brave knight") knight")

# --- --- Load Load Data Data and and Initialize Initialize ChromaDB ChromaDB --- ---
@st.cache_resource
def load_chromadb_collection(): load_chromadb_collection():
    # Adjust Adjust the the path path as as needed needed
    client chromadb.PersistentClient(r"D:\Christ\T4\LLM\Lab = 2") chromadb.PersistentClient(r"D:\Christ\T4\LLM\Lab 2")
    collection = = client.get_or_create_collection("folk_stories") client.get_or_create_collection("folk_stories")
    return client, client, collection collection

client, collection collection = = load_chromadb_collection() load_chromadb_collection()

@st.cache_data
def load_text_data(): load_text_data():
    # Adjust Adjust the the path path as as needed needed
    df = = pd.read_csv(r"D:\Christ\T4\LLM\Lab pd.read_csv(r"D:\Christ\T4\LLM\Lab 2\data\1000Folk_Story_around_the_Globe.csv") 2\data\1000Folk_Story_around_the_Globe.csv")
    story_seq = = ['genre', ['genre', 'title', 'title', 'full_text'] 'full_text']
    def preprocess_text(text): preprocess_text(text):
        text = = text.replace('\n', text.replace('\n', ' ' ').replace('\r', ').replace('\r', ' ' ') ')
        text = = ' ' '.join(text.split()) '.join(text.split())
        return text text
    df['full_text'] = = df['full_text'].apply(preprocess_text) df['full_text'].apply(preprocess_text)
    df = = df[story_seq] df[story_seq]
    return df df

text_data = = load_text_data() load_text_data()

# --- --- Helper Helper Functions Functions --- ---
def rag_prompt(topic, rag_prompt(topic, length length==500):500):
    """
    Generate a a prompt prompt for for a a text text generation generation task. task.
    """
    prompt = = f"Write f"Write a a {length} {length}--wordword story story on on the the topic topic of of {topic}. {topic}. " "
    return prompt prompt

def query_ollama(prompt, query_ollama(prompt, model model=="gemma:2b","gemma:2b", temperature temperature==0.7,0.7, max_tokens max_tokens==500):500):
    """
    Query the the Ollama Ollama API API for for text text generation. generation.
    """
    url = = "http://localhost:11434/api/generate" "http://localhost:11434/api/generate"
    payload = = { {
        "model": model, model,
        "prompt": prompt, prompt,
        "stream": False, False,
        "options": { {
            "temperature": temperature, temperature,
            "num_predict": max_tokens max_tokens
        }
    }
    try:
        response = = requests.post(url, requests.post(url, json json==payload)payload)
        response.raise_for_status()
        return response.json().get("response", response.json().get("response", "") "")
    except requests.exceptions.HTTPError requests.exceptions.HTTPError as as e: e:
        st.error(f"HTTPError: {e}\nIs {e}\nIs the the Ollama Ollama server server running running and and is is the the model model available?") available?")
        return None None
    except Exception Exception as as e: e:
        st.error(f"Error: {e}") {e}")
        return None None

def retrieve_stories(query, retrieve_stories(query, n_results n_results==5):5):
    """
    Retrieve relevant relevant stories stories from from ChromaDB ChromaDB for for a a given given query. query.
    """
    results = = collection.query( collection.query(
        query_texts==[query],[query],
        n_results==n_resultsn_results
    )
    stories = = [] []
    for doc, doc, meta meta in in zip(results['documents'][0], zip(results['documents'][0], results['metadatas'][0]): results['metadatas'][0]):
        stories.append({
            "title": meta.get('title', meta.get('title', 'Unknown'), 'Unknown'),
            "genre": meta.get('genre', meta.get('genre', 'Unknown'), 'Unknown'),
            "text": doc doc
        })
    return stories stories

# --- --- Main Main Logic Logic --- ---
if st.button("Get st.button("Get Stories"): Stories"):
    with st.spinner("Retrieving st.spinner("Retrieving similar similar stories..."): stories..."):
        retrieved_stories = = retrieve_stories(user_query, retrieve_stories(user_query, n_results n_results==3)3)
        st.subheader("Similar Stories Stories from from Database") Database")
        for idx, idx, story story in in enumerate(retrieved_stories, enumerate(retrieved_stories, 1): 1):
            with st.expander(f"{idx}. st.expander(f"{idx}. {story['title']} {story['title']} ({story['genre']})"): ({story['genre']})"):
                st.write(story['text'][:1000] + + ("..." ("..." if if len(story['text']) len(story['text']) > > 1000 1000 else else "")) ""))

    with st.spinner("Generating st.spinner("Generating a a new new story story with with Ollama..."): Ollama..."):
        prompt = = rag_prompt(user_query, rag_prompt(user_query, length length==max_length)max_length)
        ollama_response = = query_ollama(prompt, query_ollama(prompt, model model==selected_model,selected_model, temperature temperature==temperature,temperature, max_tokens max_tokens==max_length)max_length)
        if ollama_response: ollama_response:
            st.subheader("Generated Story") Story")
            st.write(ollama_response)
        else:
            st.warning("No response response from 

st.markdown("-----
st.markdown("**Note:** Make Make sure sure Ollama Ollama server server is is running running and and the the model model is is available available at at http://localhost:11434") http://localhost:11434")-")")from Ollama.") Ollama.")